# Link Prediction and Retrieval on Graphs  with GCN model


In [1]:
pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.1/63.1 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 28.1 MB/s eta 0:00:0000:01
Note: you may need to restart the kernel to use updated packages.


In [2]:
import time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from collections import defaultdict
from torch_geometric.utils import to_undirected, negative_sampling, subgraph

#  CoraFull Complete Dataset

In [3]:
from torch_geometric.datasets import CoraFull

dataset = CoraFull(root='path/to/data/CoraFull')
data = dataset[0]

edge_index = to_undirected(data.edge_index) # directed or undirected?;  undirected

num_edges = edge_index.size(1)
num_nodes = data.num_nodes

train_size = int(0.6* num_edges)
test_size = num_edges - train_size

train_edge_index = edge_index[:, :train_size]
test_edge_index = edge_index[:, train_size:]

train_nodes= train_edge_index.reshape(-1).unique()
test_nodes = test_edge_index.reshape(-1).unique()

print("CoraFull ",data) # how do we make use of y; no need here
print(f'nodes: {data.num_nodes};   features per node: {data.num_node_features};  classes: {dataset.num_classes};')
print()
print(f'edges: {data.num_edges}    train_edges: {train_edge_index.size(1)}      test_edges: {test_edge_index.size(1)} ')
print(f"train_nodes: {train_nodes.size(0)}     test_nodes: {test_nodes.size(0)}")

Processing...
Done!


CoraFull  Data(x=[19793, 8710], edge_index=[2, 126842], y=[19793])
nodes: 19793;   features per node: 8710;  classes: 70;

edges: 126842    train_edges: 76105      test_edges: 50737 
train_nodes: 17426     test_nodes: 16178


#  set Q of 1000 nodes such that each node is part of atleast one triangle

In [4]:
def get_connected_nodes(node_id, our_edge_index):
    mask_u = our_edge_index[0] == node_id
    mask_v = our_edge_index[1] == node_id
    neighbors_u = our_edge_index[1][mask_u] 
    neighbors_v = our_edge_index[0][mask_v]  
    connected_nodes = torch.cat([neighbors_u, neighbors_v])
    connected_nodes = torch.unique(connected_nodes)
    return connected_nodes

#finding Q set
triangle_nodes = set() 
test_edges = test_edge_index.t() 
print("test_edges:",test_edges.size(), "test_nodes:",len(test_nodes))
start_time = time.time()
for u, v in test_edges:
    nodes_u=get_connected_nodes(u, test_edge_index)
    nodes_v=get_connected_nodes(v, test_edge_index)
    mask = torch.isin(nodes_u, nodes_v)
    intersection_nodes = nodes_u[mask]
    if intersection_nodes.size(0) > 1:
        triangle_nodes.add(u.item())
        triangle_nodes.add(v.item())
triangle_nodes_tensor = torch.tensor(list(triangle_nodes), dtype=torch.long)
Q = triangle_nodes_tensor[:1000]
end_time = time.time()
Qtime=end_time-start_time

print("triangle_nodes:",len(triangle_nodes), f"; time taken to find them:{Qtime:.8f} seconds")
print("(test nodes that are part of atleast one triangle in test induced subgraph)")
print()
print(f"Q set, first {len(Q)} nodes of triangle_nodes")
print("Q:",Q)

test_edges: torch.Size([50737, 2]) test_nodes: 16178
triangle_nodes: 4835 ; time taken to find them:27.53096843 seconds
(test nodes that are part of atleast one triangle in test induced subgraph)

Q set, first 1000 nodes of triangle_nodes
Q: tensor([ 8192, 16386,  8195,  8196,  8197, 16388, 16389, 16392, 16393, 16390,
            9, 16396,    12, 16398,  8207,  8208, 16401, 16402, 16403,  3260,
        16405,  8213, 16407,  8214, 16406,  8218, 16408, 16404,  8221,    30,
        16415,    31, 16409, 16410, 16420, 16422,    39, 16423, 16425, 16426,
        16427, 16424, 16431, 16432, 16433, 16434, 16436, 16439, 16441, 16446,
        16449, 16450,  8259, 16451,    68, 16454, 16453, 16456, 16457, 16458,
        16459,    76, 16452, 16455, 16466, 16467,    82,    85, 16471, 16473,
        16474,  8282, 16476,    91, 16478, 16481, 16482, 16483, 16484, 16486,
        16491, 16492, 19560, 16494, 16495, 16493, 16498, 16499, 16500, 19562,
        16503, 16504,  8313, 16511, 16512, 16513, 16515,

# **GCN TRAINING**

In [5]:
from torch_geometric.nn import GCNConv

class GCN(torch.nn.Module):
    def __init__(self, in_channels, hidden_channels, out_channels):
        super(GCN, self).__init__()
        self.conv1 = GCNConv(in_channels, hidden_channels)
        self.conv2 = GCNConv(hidden_channels, out_channels)
    def forward(self, x, edge_index):
        x = self.conv1(x, edge_index)
        x = F.relu(x)
        x = self.conv2(x, edge_index)
        return x
    
def auc_loss(embeddings, pos_edge_index, neg_edge_index):
    pos_scores = F.cosine_similarity(embeddings[pos_edge_index[0]], embeddings[pos_edge_index[1]], dim=1) #[76105]; embeddings[pos_edge_index[-]]: [76105, 32]
    neg_scores = F.cosine_similarity(embeddings[neg_edge_index[0]], embeddings[neg_edge_index[1]], dim=1)
    return F.relu(1.0 + neg_scores - pos_scores).mean()


epochs=250
gcnmodel = GCN(in_channels=data.num_node_features, hidden_channels=64, out_channels=32)
optimizer = torch.optim.Adam(gcnmodel.parameters(), lr=0.01)

In [6]:
positive_edge_set = set()
for i in range(train_edge_index.size(1)):
    u, v = train_edge_index[0, i].item(), train_edge_index[1, i].item()
    positive_edge_set.add(frozenset([u, v])) 

def negative_sampling_fix(valid_neg_edge_index, t):
    while len(valid_neg_edge_index) < t:
        neg_edge_index1 = negative_sampling(
            train_edge_index,
            train_nodes.size(0),
            train_edge_index.size(1)
        )
        for i in range(neg_edge_index1.size(1)):
            u, v = neg_edge_index[0, i].item(), neg_edge_index[1, i].item()
            if frozenset([u, v]) not in positive_edge_set:
                if((u,v) not in valid_neg_edge_index):
                    valid_neg_edge_index.append((u,v))    
                if (v,u) not in valid_neg_edge_index:
                    valid_neg_edge_index.append((v, u))  
    
    for u, v in valid_neg_edge_index:
        connected_u = get_connected_nodes(u, train_edge_index)
        connected_v = get_connected_nodes(v, train_edge_index)
        if v in connected_u or u in connected_v:
            print(f"ISSUE NOT RESOLVED")
            
    print("issue in negative sampling is SORTED now")       
    return valid_neg_edge_index
     

for epoch in range(epochs):
    gcnmodel.train()
    optimizer.zero_grad()
    embeddings = gcnmodel(data.x, edge_index)
    valid_neg_edge_index = []
    neg_edge_index = negative_sampling(
        train_edge_index,
        train_nodes.size(0),
        train_edge_index.size(1)
    )
    # # for i in range(neg_edge_index.size(1)):
    # #     u, v = neg_edge_index[0, i].item(), neg_edge_index[1, i].item()
    # #     if frozenset([u, v]) in positive_edge_set:
    # #         print(f"({u}, {v}) from negative_sampling is INVALID")
            
    # for i in range(neg_edge_index.size(1)):
    #     if(len(valid_neg_edge_index)>= neg_edge_index.size(1)):
    #         break
    #     u, v = neg_edge_index[0, i].item(), neg_edge_index[1, i].item()
    #     if frozenset([u, v]) not in positive_edge_set:
    #         valid_neg_edge_index.append((u, v)) 
    #         if(len(valid_neg_edge_index)>= neg_edge_index.size(1)):
    #             break
    #     else:
    #         print(f"({u}, {v}) from negative_sampling is INVALID")
    
    # if(len(valid_neg_edge_index) < neg_edge_index.size(1)):
    #      # print("negative_sampling_fix NEEDED")
    #      valid_neg_edge_index = negative_sampling_fix(valid_neg_edge_index, train_edge_index.size(1))
    #      # print("negative_sampling_fix DONE")
    # else:
    #     # print("negative_sampling_fix NOT NEEDED")
    # valid_neg_edge_index = torch.tensor(valid_neg_edge_index).T

    valid_neg_edge_index = neg_edge_index # uncomment above and remoev me
    pos_edge_index=train_edge_index
    loss = auc_loss(embeddings, pos_edge_index, valid_neg_edge_index )
    loss.backward()
    optimizer.step()
    
    if epoch % 10 == 0:
        print(f'Epoch: {epoch}, Loss: {loss:.6f}')

Epoch: 0, Loss: 0.621373
Epoch: 10, Loss: 0.357592
Epoch: 20, Loss: 0.307041
Epoch: 30, Loss: 0.273452
Epoch: 40, Loss: 0.263484
Epoch: 50, Loss: 0.252026
Epoch: 60, Loss: 0.231384
Epoch: 70, Loss: 0.211526
Epoch: 80, Loss: 0.194671
Epoch: 90, Loss: 0.184642
Epoch: 100, Loss: 0.172815
Epoch: 110, Loss: 0.166241
Epoch: 120, Loss: 0.160186
Epoch: 130, Loss: 0.155352
Epoch: 140, Loss: 0.151260
Epoch: 150, Loss: 0.145942
Epoch: 160, Loss: 0.144295
Epoch: 170, Loss: 0.142314
Epoch: 180, Loss: 0.138684
Epoch: 190, Loss: 0.138495
Epoch: 200, Loss: 0.134426
Epoch: 210, Loss: 0.131399
Epoch: 220, Loss: 0.132610
Epoch: 230, Loss: 0.129771
Epoch: 240, Loss: 0.125873


In [7]:
gcnmodel.eval()
with torch.no_grad():
    all_embeddings = gcnmodel(data.x, data.edge_index)

test_embeddings = all_embeddings[test_nodes]
qtest_embeddings = all_embeddings[Q]

print("test_embeddings:", test_embeddings.size())
print("qtest_embeddings:", qtest_embeddings.size())

test_embeddings: torch.Size([16178, 32])
qtest_embeddings: torch.Size([1000, 32])


# **GCN INFERENCE**

In [8]:
def get_top_k_candidates(test_embeddings, all_embeddings, Q, total_time_Q):  
    l=0
    top_k_results = {}
    for q in Q:
        l=l+1
        if(l%100==0):
            print(l)
        start_time = time.time()
        q_embedding = all_embeddings[q].unsqueeze(0)
        all_scores = F.cosine_similarity(q_embedding, test_embeddings, dim=1)
        if q in test_nodes: 
            q_index = (test_nodes == q).nonzero(as_tuple=True)[0]
            all_scores[q_index] = float('-inf')
        top_nodes = [(test_nodes[i].item(), all_scores[i].item()) for i in all_scores.argsort(descending=True)]
        top_k_results[q] = top_nodes[:5000]
        end_time = time.time()
        total_time_Q += end_time - start_time
    return top_k_results, total_time_Q

top_k_results, total_time_Q = get_top_k_candidates(test_embeddings, all_embeddings, Q, 0)
total_time_Q = total_time_Q / 1000

100
200
300
400
500
600
700
800
900
1000


In [9]:
precision_results = {}
top_k_list = [1, 5, 10] 

for q_node, candidates in top_k_results.items():
    all_connected_nodes = get_connected_nodes(q_node, test_edge_index)
    precision_results[q_node] = {}
    mrr_sum = 0.0
    
    for k in top_k_list:
        nodes = candidates[:k]
        top_k_nodes = [node for node, score in nodes]
        top_k_tensor = torch.tensor(top_k_nodes)
        intersected_nodes = top_k_tensor[torch.isin(top_k_tensor, all_connected_nodes)]
        precision_at_k = len(intersected_nodes) / k
        precision_results[q_node][f"P@{k}"] = precision_at_k

    first_relevant_rank = None
    top_nodes = [node for node, score in candidates]
    for i, node in enumerate(top_nodes):
        if node in all_connected_nodes:
            first_relevant_rank = i + 1
            break
    if first_relevant_rank is not None:
        mrr_sum += 1 / first_relevant_rank
    precision_results[q_node]["MRR"] = mrr_sum

total_mrr = sum(result["MRR"] for result in precision_results.values())
average_mrr = total_mrr / len(precision_results) if precision_results else 0
precision_results["average_MRR"] = average_mrr
sum_p1, sum_p5, sum_p10, sum_mrr = 0, 0, 0, 0
num_results = len(precision_results)

for key, value in precision_results.items():
    if isinstance(value, dict):
        sum_p1 += value.get('P@1', 0)
        sum_p5 += value.get('P@5', 0)
        sum_p10 += value.get('P@10', 0)
        sum_mrr += value.get('MRR', 0)

average_p1 = sum_p1 / num_results
average_p5 = sum_p5 / num_results
average_p10 = sum_p10 / num_results
average_mrr = sum_mrr / num_results

print(f"(CoraFull dataset, GCN model):   GCN INFERENCE")
print()
print(f"P@1: {average_p1}, P@5: {average_p5}, P@10: {average_p10}, MRR: {average_mrr}")
print(f"Average retrieval time: {total_time_Q} seconds")

(CoraFull dataset, GCN model):   GCN INFERENCE

P@1: 0.5704295704295704, P@5: 0.42017982017982014, P@10: 0.3158841158841166, MRR: 0.6966583019843102
Average retrieval time: 0.15776622796058654 seconds


# **FASTER INFERENCE - Random projection hashing**

In [10]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

num_hyperplanes_list = [2,4,6]   # change accordingly ......
k_values=[1,5,10]

In [11]:
def generate_random_hyperplanes(num_hyperplanes, feature_dim):
    return torch.randn(num_hyperplanes, feature_dim).to(device)

def compute_hash_codes(features, hyperplanes):
    hash_codes = features.matmul(hyperplanes.T) > 0  
    return [''.join(code.cpu().numpy().astype(str)) for code in hash_codes.int()]

def run_random_projection_pipeline(qtest_embeddings, test_embeddings, num_hyperplanes):
    feature_dim = test_embeddings.shape[1]
    hyperplanes = generate_random_hyperplanes(num_hyperplanes, feature_dim)
    
    test_hash_codes = compute_hash_codes(test_embeddings.to(device), hyperplanes)
    qtest_hash_codes = compute_hash_codes(qtest_embeddings.to(device), hyperplanes)
    
    precision_at_k = {k: [] for k in k_values}
    retrieval_times=0
    mrr_at_queries = []
    for i, qtest_code in enumerate(qtest_hash_codes):
        mrr_sum = 0.0
        start_time = time.time()
        matching_indices = [j for j, test_code in enumerate(test_hash_codes) if test_code == qtest_code]
        if matching_indices:
            samebucket_embeddings = test_embeddings[matching_indices]
            similarities = torch.nn.functional.cosine_similarity(qtest_embeddings[i].unsqueeze(0), samebucket_embeddings).cpu()
            ranked_indices = torch.argsort(similarities, descending=True)
            top_k_node_ids = [test_nodes[matching_indices[idx]] for idx in ranked_indices]
            top_k_node_ids = [node_id for node_id in top_k_node_ids if node_id != Q[i]]
            end_time = time.time() 
            retrieval_times=retrieval_times+end_time - start_time
            connected = get_connected_nodes(Q[i],test_edge_index)

            for k in k_values:
                top_k_retrieved = torch.tensor(top_k_node_ids[:k])
                mask = torch.isin(connected, top_k_retrieved)
                intersected_nodes = connected[mask]
                precision = len(intersected_nodes) / k
                precision_at_k[k].append(precision)
                
            first_relevant_rank = None
            for idx, node in enumerate(top_k_node_ids):
                if node in connected:
                    first_relevant_rank = idx + 1
                    break
            if first_relevant_rank is not None:
                mrr_sum += 1 / first_relevant_rank
            mrr_at_queries.append(mrr_sum)
            
    average_mrr = sum(mrr_at_queries) / len(mrr_at_queries) if mrr_at_queries else 0
    average_retrieval_time = retrieval_times/ 1000    
    average_precision_at_k = {k: sum(precision_at_k[k]) / len(precision_at_k[k]) if precision_at_k[k] else 0 for k in k_values}
    return average_mrr, average_precision_at_k, average_retrieval_time

print(f"(CoraFull dataset, GCN model):   FASTER INFERENCE - Random projection hashing")
print()
for num_hyperplanes in num_hyperplanes_list:
    average_mrr, precision_k_mean, average_retrieval_time = run_random_projection_pipeline(qtest_embeddings, test_embeddings, num_hyperplanes)
    results = {
        'n_hyperplanes': num_hyperplanes,
        'P@1': precision_k_mean.get(1, 0),
        'P@5': precision_k_mean.get(5, 0),
        'P@10': precision_k_mean.get(10, 0),
        'MRR': average_mrr,
        'Average retrieval time':average_retrieval_time
    }
    print(f"{results}")

(CoraFull dataset, GCN model):   FASTER INFERENCE - Random projection hashing

{'n_hyperplanes': 2, 'P@1': 0.557, 'P@5': 0.40259999999999996, 'P@10': 0.2986000000000005, 'MRR': 0.6821364313451589, 'Average retrieval time': 0.06426516771316529}
{'n_hyperplanes': 4, 'P@1': 0.545, 'P@5': 0.38539999999999924, 'P@10': 0.2820999999999997, 'MRR': 0.6655620070280595, 'Average retrieval time': 0.01882152009010315}
{'n_hyperplanes': 6, 'P@1': 0.523, 'P@5': 0.36499999999999944, 'P@10': 0.2579999999999999, 'MRR': 0.6408088973925419, 'Average retrieval time': 0.008074583530426026}


# **FASTER INFERENCE - Neural LSH**

In [13]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

class HashNetworkV2(nn.Module):
    def __init__(self, dim_input: int, dim_output: int):
        super().__init__()
        self.linear = nn.Linear(dim_input, dim_output)
    def _mlp(self, X: torch.Tensor) -> torch.Tensor:
        return self.linear(X)
    def forward(self, X: torch.Tensor) -> torch.Tensor:
        return F.tanh(self._mlp(X))
    def get_binary_hash(self, X: torch.Tensor) -> torch.Tensor:
        return torch.sign(self.forward(X))

In [14]:
def graph_based_loss(model, data_batch, neg_edge_index, alpha, beta, gamma):
    hash_codes = model(data_batch)
    bit_balance = torch.abs(torch.sum(hash_codes, dim=1, keepdim=True))
    bit_balance = torch.mean(bit_balance)
    sitting_on_fence = torch.sum(torch.abs(torch.abs(hash_codes) - 1), dim=1)
    sitting_on_fence = torch.mean(sitting_on_fence)
    weak_supervision = 0
    for i in range(neg_edge_index.size(1)):  
        u, v = neg_edge_index[0, i].item(), neg_edge_index[1, i].item() 
        weak_supervision += torch.dot(hash_codes[u], hash_codes[v]) 
    weak_supervision /= neg_edge_index.size(1) 
    loss = alpha * bit_balance + beta * sitting_on_fence + gamma * weak_supervision
    print(alpha * bit_balance.item(), beta * sitting_on_fence.item(), gamma *  weak_supervision.item(), loss.item())
    return loss

def train_hashing_model(test_embeddings, model, optimizer, num_epochs, alpha, beta, gamma, data):
    model.to(device)
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        data_batch = test_embeddings.clone().detach().float().to(device)
        neg_edge_index = negative_sampling(
            test_edge_index,
            test_nodes.size(0),
            test_edge_index.size(1)
        )
        loss = graph_based_loss(model, data_batch, neg_edge_index, alpha, beta, gamma)
        if not isinstance(loss, torch.Tensor):
            raise ValueError("Loss is not a tensor, expected a tensor.")
        loss.backward() 
        optimizer.step() 
    return model

In [15]:
num_epochs = 10
torch.cuda.empty_cache()
alpha = 0.4
beta = 0.2
gamma = 0.4

input_dim = test_embeddings.shape[1]  
output_dim = input_dim 
model = HashNetworkV2(input_dim, output_dim).to(device)
optimizer = optim.Adam(model.parameters(), lr=0.001)
print("next train the hash model---")
hashing_model=train_hashing_model(test_embeddings, model, optimizer, num_epochs, alpha, beta, gamma, data)
print("trainig done:::")

next train the hash model---
0.5605771541595459 4.250575637817383 0.09136311411857606 4.902515411376953
0.5032185554504395 4.248266983032226 0.08156801462173463 4.833054065704346
0.44736852645874026 4.245337677001953 0.08522552251815796 4.777932167053223
0.394328236579895 4.241777420043945 0.08517411351203918 4.721279621124268
0.34489164352416996 4.237807083129883 0.08641481399536133 4.669113636016846
0.300200366973877 4.23334083557129 0.0823448657989502 4.615886211395264
0.2624268770217896 4.228182601928711 0.07792759537696839 4.56853723526001
0.23436589241027833 4.222223281860352 0.08498221635818481 4.541571617126465
0.21674137115478517 4.215211868286133 0.08258590698242188 4.5145392417907715
0.2052431344985962 4.2070762634277346 0.07948217988014222 4.491801738739014
trainig done:::


In [16]:
import torch
from collections import defaultdict

class Hashing:
    def __init__(self, J: int, hash_dim: int, num_hash_tables: int, device: torch.device):
        self.hash_dim = min(hash_dim, 33)  # limit hash_dim to a maximum of 33
        self.num_hash_tables = num_hash_tables
        self.hash_tables = [defaultdict(list) for _ in range(num_hash_tables)]
        self.J = J  # number of bits selected for each hash table
        self.selected_positions_list = []  # to store the selected positions for each hash table  
        self.device = device
        torch.manual_seed(42)

    def create_hash_tables(self, binary_test_hash_codes: torch.Tensor, test_nodes):
        num_samples = binary_test_hash_codes.size(0)
        if self.J > self.hash_dim:
            raise ValueError(f"J-(Number of bits selected for each hash table):{self.J} cannot be larger than the hash dimension ({self.hash_dim})")
        for l in range(self.num_hash_tables):  
            if len(self.selected_positions_list) > l:  # check if positions for this hash table are already generated
                selected_positions = self.selected_positions_list[l]
            else:           
                selected_positions = torch.randperm(self.hash_dim)[:self.J]  # randomly select J bit positions for this hash table
                self.selected_positions_list.append(selected_positions)   
            for i in range(num_samples):
                projected_code = binary_test_hash_codes[i][selected_positions]  # project the hash code to the selected bit positions
                projected_code_binary = (projected_code > 0).int()  # convert floats to binary (0 or 1)
                bucket_index = torch.sum(projected_code_binary * (2 ** torch.arange(self.J - 1, -1, -1, device=projected_code_binary.device))) 
                self.hash_tables[l][bucket_index.item()].append(test_nodes[i])
            
        print("created hash_tables")

    def retrieve_images(self, test_hash_code: torch.Tensor):
        all_retrieved_indices = torch.tensor([], dtype=torch.long, device=self.device)  
        for l in range(self.num_hash_tables):  
            selected_positions = self.selected_positions_list[l]
            projected_code_binary = torch.zeros(self.J, dtype=torch.int32, device=self.device)
            selected_bits = test_hash_code[selected_positions]  # Shape: [J] where J = 5
            projected_code_binary = (selected_bits > 0).int()  # Convert to binary
            bucket_index = torch.sum(
                projected_code_binary * (2 ** torch.arange(self.J - 1, -1, -1, device=self.device))
            ).item()  # Get the bucket index as a Python scalar
            if bucket_index in self.hash_tables[l]:  # Compare using Python integer
                similar_indices = torch.tensor(self.hash_tables[l][bucket_index], dtype=torch.long, device=self.device)
                all_retrieved_indices = torch.cat((all_retrieved_indices, similar_indices))  # Concatenate retrieved indices
            else:
                print(f"No match found for bucket {bucket_index}")
        all_retrieved_indices = torch.unique(all_retrieved_indices)  # Remove duplicates
        all_retrieved_indices = all_retrieved_indices.sort().values  # Sort indices
        return all_retrieved_indices

In [17]:
test_embeddings = test_embeddings.to(device)
qtest_embeddings = qtest_embeddings.to(device)

num_hash_tab = 6
J =5 # number of bits selected for each hash table

hashing_instance = Hashing(J, hashing_model.linear.out_features, num_hash_tab, device)

print("hashing instance created")
binary_test_hash_codes = hashing_model.get_binary_hash(test_embeddings).detach().to(device)  # Get binary hash codes
print("hashcodes ready for test set, buckets okayyyyyy ", binary_test_hash_codes.size())
# print(hash_codes)

hashing_instance.create_hash_tables(binary_test_hash_codes, test_nodes)
print("HASHTABLESSSS CREATED")

binary_qtest_hash_codes = hashing_model.get_binary_hash(qtest_embeddings).detach().to(device)
print("test hash codes ready ", binary_qtest_hash_codes.size())
print("---------------------------------------------")


hashing instance created
hashcodes ready for test set, buckets okayyyyyy  torch.Size([16178, 32])
created hash_tables
HASHTABLESSSS CREATED
test hash codes ready  torch.Size([1000, 32])
---------------------------------------------


In [18]:
results=[]
times=[]
precision_at_k = {1: [], 5: [], 10: []}
mrr_values = []
for i, test_node in enumerate(Q):
    if i % 100 == 0:
        print(i)
    
    start_time = time.time()
    retrieved_nodes = hashing_instance.retrieve_images(binary_qtest_hash_codes[i])
    end_time = time.time()
    elapsed_time = end_time - start_time
    times.append(elapsed_time)
    
    device = retrieved_nodes.device 
    all_embeddings = all_embeddings.to(device) 
    retrieved_embeddings = all_embeddings[retrieved_nodes]
    qtest_embedding = all_embeddings[Q[i]].unsqueeze(0).to(device)
    
    similarities = torch.nn.functional.cosine_similarity(qtest_embedding, retrieved_embeddings)
    sorted_sim_indices = torch.argsort(similarities, descending=True)
    sorted_retrieved_nodes = retrieved_nodes[sorted_sim_indices]
    sorted_retrieved_nodes = [node_id for node_id in sorted_retrieved_nodes if node_id != Q[i]]
    connected_nodes = get_connected_nodes(Q[i], test_edge_index)
    
    for k in [1, 5, 10]:
        top_k_retrieved = sorted_retrieved_nodes[:k]
        top_k_retrieved = torch.tensor(top_k_retrieved, device=device)
        connected_nodes = connected_nodes.to(device)
        mask = torch.isin(top_k_retrieved, connected_nodes)
        intersected = top_k_retrieved[mask]
        precision_at_k[k].append(len(intersected) / k)
   
    first_relevant_rank = None
    for rank, node in enumerate(sorted_retrieved_nodes, 1): 
        if node in connected_nodes:
            first_relevant_rank = rank
            break
    if first_relevant_rank is not None:
        mrr_values.append(1 / first_relevant_rank)
    else:
        mrr_values.append(0)

average_precision_at_k = {k: sum(precision_at_k[k]) / len(precision_at_k[k]) if precision_at_k[k] else 0 for k in [1, 5, 10]}
average_mrr = sum(mrr_values) / len(mrr_values) if mrr_values else 0
average_retrieval_time = sum(times) / len(times) if times else 0

0
100
200
300
400
500
600
700
800
900


In [19]:
print(f"(CoraFull dataset, GCN model):   FASTER INFERENCE - NEURAL LSH")
print()
print(f"P@1: {average_precision_at_k[1]}, P@5: {average_precision_at_k[5]}, P@10: {average_precision_at_k[10]}, MRR: {average_mrr}")
print(f"Average retrieval time: {average_retrieval_time} seconds")

(CoraFull dataset, GCN model):   FASTER INFERENCE - NEURAL LSH

P@1: 0.57, P@5: 0.4201999999999999, P@10: 0.31580000000000064, MRR: 0.697051367161806
Average retrieval time: 0.006876643180847168 seconds
